In [1]:
import numpy as np
import torch
import torch.nn
import torch.optim
from abc import abstractmethod
from collections import defaultdict
from functools import lru_cache
from itertools import count
from typing import List, Dict
from typing import Tuple, Any
from sklearn import ensemble
from sklearn.metrics import mean_squared_error
from torch.nn import MSELoss, ReLU, L1Loss
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [2]:
from importlib import reload
from config import Config
import config
from data import MDataset, Graph, load_graphs, save_dataset_pkl, load_dataset_pkl, save_scalers_pkl, load_scalers_pkl
import data
from base_module import MModule, nested_detach
import base_module
from executor import single_train_loop, grid_search_loop
import executor
from objects import ModelType
import objects
from metric import MetricUtil
import metric
from logger import init_logging
import logger
reload(config)
reload(data)
reload(base_module)
reload(executor)
reload(objects)
reload(metric)
reload(logger)
from config import Config
from data import MDataset, Graph, load_graphs
from base_module import MModule
from objects import ModelType
from metric import MetricUtil
from logger import init_logging
init_logging()

datasets_path: /root/guohao/repos/DLT-perf-model/datasets
configs_path: /root/guohao/repos/DLT-perf-model/notebooks/configs
datasets_path: /root/guohao/repos/DLT-perf-model/datasets
configs_path: /root/guohao/repos/DLT-perf-model/notebooks/configs


In [3]:
dataset_environment_str = "RTX2080Ti_CPUALL"
normalizer_cls = StandardScaler # MinMaxScaler
dummy = False

In [21]:
train_configs = {
    ModelType.MLP.name: Config.from_dict({
        "model": "MLP",
        "all_seed": 42,
        "dataset_environment_str": dataset_environment_str,
        "dataset_normalization": "Standard",
        "dataset_params": {
            "duration_summed": False,
        },
        "dataset_dummy": False,
        "batch_size": 32,
        "eval_steps": 10000,
        "learning_rate": 1e-4,
        "epochs": 10,
        "optimizer": "Adam",
        "meta_configs": {
            "learning_rate": 0.005,
            "meta_learning_rate": 0.001,
            "meta_train_steps": 1000,
            "meta_task_per_step": 8,
            "meta_fast_adaption_step": 5,
            "meta_dataset_train_environment_strs": [dataset_environment_str],
            "meta_dataset_eval_environment_strs": [dataset_environment_str],
        },
    }),
    ModelType.PerfNet.name: Config.from_dict({
        "model": "PerfNet",
        "dataset_environment_str": dataset_environment_str,
        "dataset_normalization": "Standard",
        "all_seed": 42,
        "dataset_params": {
            "duration_summed": False,
        },
        "dataset_dummy": True,
        "batch_size": 32,
        "eval_steps": 10000,
        "learning_rate": 1e-4,
        "epochs": 10,
        "optimizer": "Adam",
        "meta_configs": {
            "learning_rate": 0.005,
            "meta_learning_rate": 0.001,
            "meta_train_steps": 1000,
            "meta_task_per_step": 8,
            "meta_fast_adaption_step": 5,
            "meta_dataset_train_environment_strs": [dataset_environment_str],
            "meta_dataset_eval_environment_strs": [dataset_environment_str],
        },
    })
}

model_type = ModelType.MLP
conf = train_configs[model_type.name]

In [5]:
eval_graphs = load_graphs(dataset_environment_str,
                            train_or_eval="train",
                            use_dummy=dummy,
                            max_row=400_000)
train_graphs = load_graphs(dataset_environment_str,
                            train_or_eval="train",
                            use_dummy=dummy,
                            max_row=200_0000)

[2023-12-20 09:16:42,629] {data.py:441} INFO - Loading graphs train
[2023-12-20 09:16:42,630] {data.py:412} INFO - Loading merged.csv
[2023-12-20 09:16:42,630] {data.py:412} INFO - Loading merged.csv
[2023-12-20 09:16:43,623] {data.py:415} INFO - Loaded merged.csv, 400000 rows
[2023-12-20 09:16:44,271] {data.py:421} INFO - Loaded rand_5000.207_7.csv, 1332 rows
[2023-12-20 09:16:44,896] {data.py:421} INFO - Loaded rand_1000.146_7.csv, 1366 rows
[2023-12-20 09:16:46,070] {data.py:421} INFO - Loaded rand_5500.22_7.csv, 2609 rows
[2023-12-20 09:16:46,959] {data.py:421} INFO - Loaded rand_4500.22_7.csv, 1996 rows
[2023-12-20 09:16:47,055] {data.py:421} INFO - Loaded vgg16.85_7.csv, 173 rows
[2023-12-20 09:16:47,211] {data.py:421} INFO - Loaded resnet18.138_7.csv, 315 rows
[2023-12-20 09:16:47,532] {data.py:421} INFO - Loaded rand_2000.251_7.csv, 683 rows
[2023-12-20 09:16:48,127] {data.py:421} INFO - Loaded rand_6500.11_7.csv, 1302 rows
[2023-12-20 09:16:49,433] {data.py:421} INFO - Loaded 

In [6]:
len(eval_graphs), len(train_graphs)

(295, 1501)

In [7]:

def init_dataset(graphs: List[Graph]) -> MDataset:
    op_X, op_Y = list(), list()
    data_idx_to_graph = dict()
    counter = iter(count())
    op_feature_len = 0

    def node_features(g: Graph) -> Tuple[
        List[Dict], List[Dict]]:
        X, Y = list(), list()
        for i, node in enumerate(g.nodes):
            x_op_feature = node.op.to_feature_array("complex")
            x = {
                "x_op_feature": x_op_feature
            }
            node_durations = (node.duration,node.gap)
            x["x_id"] = i
            x["x_graph_id"] = g.ID
            y = {"y_node_durations": node_durations, "y_id": i, "y_graph_id": g.ID}
            X.append(x)
            Y.append(y)
        return X, Y

    for graph in graphs:
        X, Y = node_features(graph)
        for x in X:
            # if len(x['x_op_feature'])!=37:
            #     print(x['x_graph_id'], len(x['x_op_feature']))
            op_feature_len = max(op_feature_len, len(x["x_op_feature"]))
        op_X.extend(X)
        op_Y.extend(Y)
        for i in range(len(X)):
            data_idx_to_graph[next(counter)] = graph
    for x in op_X:
        v = x["x_op_feature"]
        x["x_op_feature"] = np.pad(v, (0, op_feature_len - v.size))

    dataset = MDataset(op_X, op_Y)
    return dataset

train_ds = init_dataset(train_graphs)
eval_ds = init_dataset(eval_graphs)

In [8]:
def get_scalers(ds):
    scaler_cls = normalizer_cls
    op_feature_array = list()
    y_array = list()

    for data in ds:
        feature, label = data
        op_feature_array.append(feature["x_op_feature"])
        y_array.append(label["y_node_durations"])

    op_feature_array = np.array(op_feature_array)
    y_array = np.array(y_array)

    op_feature_scaler = scaler_cls()
    op_feature_scaler.fit(op_feature_array)

    y_scaler = scaler_cls()
    y_scaler.fit(y_array)
    return op_feature_scaler, y_scaler

scalers = get_scalers(train_ds)
op_feature_scaler, y_scaler = scalers

In [9]:

def preprocess_dataset(ds: MDataset) -> MDataset:
    op_feature_array = list()
    y_array = list()

    for data in ds:
        feature, label = data
        op_feature_array.append(feature["x_op_feature"])
        y_array.append(label["y_node_durations"])

    op_feature_array = np.array(op_feature_array, dtype=np.float32)
    y_array = np.array(y_array, dtype=np.float32)


    op_feature_array = op_feature_scaler.transform(op_feature_array)
    y_array = y_scaler.transform(y_array)

    processed_features = list()
    processed_labels = list()
    for i, data in enumerate(ds):
        feature, label = data
        processed_features.append({
            "x_id": feature["x_id"],
            "x_graph_id": feature["x_graph_id"],
            # 运行时再传到cuda那边
            # "x_op_feature": torch.Tensor(op_feature_array[i]).to(device=self.conf.device)
            "x_op_feature": torch.Tensor(op_feature_array[i])
        })
        processed_labels.append({
            "y_id": label["y_id"],
            "y_graph_id": label["y_graph_id"],
            # "y_node_durations": torch.Tensor(y_array[i]).to(device=self.conf.device)
            "y_node_durations": torch.Tensor(y_array[i])
        })

    ds = MDataset(processed_features, processed_labels)
    return ds

preprocessed_train_ds = preprocess_dataset(train_ds)
preprocessed_eval_ds = preprocess_dataset(eval_ds)

In [10]:
# save_dataset_pkl(preprocessed_train_ds, conf.dataset_environment, "OpBased", 'train',
#                          conf.dataset_normalization)
# save_dataset_pkl(preprocessed_eval_ds, conf.dataset_environment, "OpBased", 'eval',
#                          conf.dataset_normalization)
# save_scalers_pkl(scalers, conf.dataset_environment, "OpBased", 'train',
#                          conf.dataset_normalization)

In [11]:
# preprocessed_train_ds = load_dataset_pkl(conf.dataset_environment, "OpBased", 'train', 
#                                          conf.dataset_normalization)
# preprocessed_eval_ds = load_dataset_pkl(conf.dataset_environment, "OpBased", 'eval',
#                                         conf.dataset_normalization)
# scalers = load_scalers_pkl(conf.dataset_environment, "OpBased", 'train',
#                            conf.dataset_normalization)


In [12]:
op_feature_scaler, y_scaler = scalers

In [13]:
def compute_evaluate_metrics(input_batches, output_batches, eval_loss) -> Dict[str, float]:
    batches_len = len(input_batches)

    def compute_op_durations(_logits):
        transformed: np.ndarray = y_scaler.inverse_transform(_logits)
        durations = transformed.sum(axis=1)
        return durations

    graph_id_to_duration_pred = defaultdict(int)
    for idx in range(batches_len):
        inputs = input_batches[idx]
        logits = output_batches[idx]
        logits = nested_detach(logits)
        logits = logits.cpu().numpy()
        graph_ids = inputs["x_graph_id"]
        op_durations = compute_op_durations(logits)
        for i, graph_id in enumerate(graph_ids):
            op_duration = op_durations[i].item()
            graph_id_to_duration_pred[graph_id] += op_duration
    duration_metrics = MetricUtil.compute_duration_metrics(eval_graphs, graph_id_to_duration_pred)
    return {"eval_loss": eval_loss, **duration_metrics}


In [14]:

def to_device(conf, features, labels):
    features['x_op_feature'] = features["x_op_feature"].to(device=conf.device)
    labels['y_node_durations'] = labels['y_node_durations'].to(device=conf.device)
    return features, labels

In [15]:
class MLPModel(MModule):

    @staticmethod
    def dimension_len(t):
        return t[-1] - t[0]
    
    @staticmethod
    def grid_search_model_params() -> Dict[str, List[Any]]:
        return {
            "learning_rate": [1e-3, 1e-4, 1e-5],
            'batch_size': [16, 32, 64],
            'epochs': [5, 10, 20, 30],
            'optimizer': ['Adam', 'SGD'],
        }
        # return {}
    
    def __init__(self, input_dimension, output_dimension, **kwargs):
        super().__init__(**kwargs)
        self.input = torch.nn.Linear(input_dimension, 256)
        self.relu1 = ReLU()
        self.dense1 = torch.nn.Linear(256, 128)
        self.relu2 = ReLU()
        self.dense2 = torch.nn.Linear(128, 64)
        self.relu3 = ReLU()
        self.output = torch.nn.Linear(64, output_dimension)
        self.loss_fn = L1Loss()

    def forward(self, X):
        X = X["x_op_feature"]
        X = self.input(X)
        X = self.relu1(X)
        X = self.dense1(X)
        X = self.relu2(X)
        X = self.dense2(X)
        X = self.relu3(X)
        Y = self.output(X)
        return Y

    def compute_loss(self, outputs, Y):
        node_durations = Y["y_node_durations"]
        loss = self.loss_fn(outputs, node_durations)
        return loss


def init_MLP_model() -> MModule | Any:
    sample_preprocessed_ds = preprocessed_train_ds
    sample_x_dict = sample_preprocessed_ds.features[0]
    sample_y_dict = sample_preprocessed_ds.labels[0]
    return MLPModel(input_dimension=len(sample_x_dict["x_op_feature"]),
                    output_dimension=len(sample_y_dict["y_node_durations"]))

In [16]:

class PerfNetModel(MModule):
    @staticmethod
    def dimension_len(t):
        return t[-1] - t[0]
    
    @staticmethod
    def grid_search_model_params() -> Dict[str, List[Any]]:
        return {
            "learning_rate": [1e-3, 1e-4, 1e-5],
            'batch_size': [16, 32, 64],
            'epochs': [5, 10, 20, 30],
            'optimizer': ['Adam', 'SGD'],
        }
        # return {}

    def __init__(self, output_dimension, **kwargs):
        super().__init__(**kwargs)
        self.conv1 = torch.nn.LazyConv1d(out_channels=32, kernel_size=3, bias=True, padding_mode='zeros')
        self.conv2 = torch.nn.LazyConv1d(out_channels=128, kernel_size=2, bias=True, padding_mode='zeros')
        self.flatten = torch.nn.Flatten()
        self.dense1 = torch.nn.LazyLinear(32)
        self.relu1 = ReLU()
        self.dense2 = torch.nn.LazyLinear(64)
        self.relu2 = ReLU()
        self.dense3 = torch.nn.LazyLinear(128)
        self.relu3 = ReLU()
        self.dense4 = torch.nn.LazyLinear(256)
        self.relu4 = ReLU()
        self.dropout = torch.nn.Dropout(p=0.3)
        self.output = torch.nn.LazyLinear(output_dimension)
        self.loss_fn = L1Loss()

    def forward(self, X):
        X = X["x_op_feature"]
        X = torch.unsqueeze(X, dim=1)
        X = self.conv1(X)
        X = self.conv2(X)
        X = self.flatten(X)
        X = self.dense1(X)
        X = self.relu1(X)
        X = self.dense2(X)
        X = self.relu2(X)
        X = self.dense3(X)
        X = self.relu3(X)
        X = self.dense4(X)
        X = self.relu4(X)
        X = self.dropout(X)
        Y = self.output(X)
        return Y

    def compute_loss(self, outputs, Y):
        node_durations = Y["y_node_durations"]
        loss = self.loss_fn(outputs, node_durations)
        return loss


def init_PerfNet_model() -> MModule | Any:
    sample_y_dict = preprocessed_train_ds.labels[0]
    return PerfNetModel(output_dimension=len(sample_y_dict["y_node_durations"]))



In [17]:
init_model_funcs = {
    ModelType.MLP.name: init_MLP_model,
    ModelType.PerfNet.name: init_PerfNet_model,
}


init_model = init_model_funcs[model_type.name]

model = init_model()
model = model.to(conf.device)


/root/anaconda3/envs/DLT-pref-model/lib/python3.10/site-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


In [22]:
single_train_loop(model_type, conf, preprocessed_train_ds, preprocessed_eval_ds, model, compute_evaluate_metrics, to_device)

[2023-12-20 11:11:11,559] {executor.py:148} INFO - ModelType.MLP start single training.
[2023-12-20 11:11:11,560] {executor.py:150} INFO - ModelType.MLP training epoch 0


  0%|          | 0/62500 [00:00<?, ?it/s]

[2023-12-20 11:11:11,684] {executor.py:111} INFO - ModelType.MLP trained for 0.125396048 seconds.
[2023-12-20 11:11:11,685] {executor.py:112} INFO - ModelType.MLP eval at step 0.
[2023-12-20 11:11:24,019] {executor.py:116} INFO - ModelType.MLP train loss: 0.016828374937176704, eval metrics: {'eval_loss': 0.01839613865005318, 'MRE': 0.1559010296051249, 'MAE': 0.11380958377369707, 'RMSE': 104.96509482974797}
[2023-12-20 11:11:24,073] {executor.py:187} INFO - Saving model at step 0 with loss 0.016828374937176704,save path: /root/guohao/repos/DLT-perf-model/notebooks/ckpts/RTX2080Ti_CPUALL/MLP/single_train2023-12-20_11-11-11


 16%|█▌        | 9997/62500 [00:48<03:09, 277.38it/s]

[2023-12-20 11:12:00,213] {executor.py:111} INFO - ModelType.MLP trained for 48.654025729 seconds.
[2023-12-20 11:12:00,213] {executor.py:112} INFO - ModelType.MLP eval at step 10000.
[2023-12-20 11:12:12,457] {executor.py:116} INFO - ModelType.MLP train loss: 0.013243342749774456, eval metrics: {'eval_loss': 0.019142582029993645, 'MRE': 0.15285790712584243, 'MAE': 0.11910142708700426, 'RMSE': 108.45925683408034}
[2023-12-20 11:12:12,510] {executor.py:187} INFO - Saving model at step 10000 with loss 0.013243342749774456,save path: /root/guohao/repos/DLT-perf-model/notebooks/ckpts/RTX2080Ti_CPUALL/MLP/single_train2023-12-20_11-11-11


...

[2023-12-20 11:50:19,277] {executor.py:111} INFO - ModelType.MLP trained for 2347.718684831 seconds.
[2023-12-20 11:50:19,278] {executor.py:112} INFO - ModelType.MLP eval at step 480000.
[2023-12-20 11:50:31,206] {executor.py:116} INFO - ModelType.MLP train loss: 0.01074175164103508, eval metrics: {'eval_loss': 0.01694277050758712, 'MRE': 0.138152107230635, 'MAE': 0.10125798507709348, 'RMSE': 74.88878722578994}
[2023-12-20 11:50:31,257] {executor.py:187} INFO - Saving model at step 480000 with loss 0.01074175164103508,save path: /root/guohao/repos/DLT-perf-model/notebooks/ckpts/RTX2080Ti_CPUALL/MLP/single_train2023-12-20_11-11-11


 84%|████████▍ | 52487/62500 [04:15<00:36, 273.54it/s]

[2023-12-20 11:51:08,624] {executor.py:111} INFO - ModelType.MLP trained for 2397.065079509 seconds.
[2023-12-20 11:51:08,625] {executor.py:112} INFO - ModelType.MLP eval at step 490000.
[2023-12-20 11:51:20,749] {executor.py:116} INFO - ModelType.MLP train loss: 0.012763307429850101, eval metrics: {'eval_loss': 0.017551989438370802, 'MRE': 0.13825136079335948, 'MAE': 0.10278707953265952, 'RMSE': 79.15120289077805}
[2023-12-20 11:51:20,802] {executor.py:187} INFO - Saving model at step 490000 with loss 0.012763307429850101,save path: /root/guohao/repos/DLT-perf-model/notebooks/ckpts/RTX2080Ti_CPUALL/MLP/single_train2023-12-20_11-11-11


 93%|█████████▎| 58089/62500 [04:48<00:21, 201.42it/s]


KeyboardInterrupt: 

In [ ]:
from executor import grid_search_loop
# reload(executor)
# grid_search_loop(
#     model_type,
#     conf,
#     preprocessed_train_ds,
#     preprocessed_eval_ds,
#     compute_evaluate_metrics,
#     to_device,
#     init_model,
# )